In [1]:
import pandas as pd
from pha_tools.io import gather_data_filenames, load_donation_data_from_filenames

# Data Ingestion

In [2]:
data_dir = '../data/'
glob_text = 'transactions*.xlsx'
filenames = gather_data_filenames(data_dir, glob_text)
donations = load_donation_data_from_filenames(filenames)

In [3]:
years = sorted(int(yr) for yr in donations['date'].dt.year.unique())

In [4]:
def normalize_name(name):
    return ' '.join(s.capitalize() for s in name.split())

In [5]:
donations['donor_name'] = donations['donor_name'].apply(normalize_name)
donations['year'] = donations.date.dt.year

# Year-centric Analysis (loops)

In [6]:
donors_by_year = {}
for year in years:
    donors_by_year[year] = set(donations['donor_name'].loc[donations['date'].dt.year == year])

In [7]:
report = {}
for last_year in years:
    this_year = last_year + 1
    if this_year not in years: break
    this_year_donations = donations.loc[donations['date'].dt.year == this_year]
    last_year_donations = donations.loc[donations['date'].dt.year == last_year]
    
    report[this_year] = {}
    last_year_donors = donors_by_year[last_year]
    this_year_donors = donors_by_year[this_year]
    new = this_year_donors - last_year_donors
    report[this_year]['new'] = new

    lost = last_year_donors - this_year_donors
    report[this_year]['lost'] = lost

    returning = this_year_donors & last_year_donors
    report[this_year]['returning'] = returning

    report[this_year]['upgrades'] = {}  # name: (this_year_sum, last_year_sum)
    report[this_year]['downgrades'] = {}  # name: (this_year_sum, last_year_sum)
    for name in returning:
        this_year_sum = this_year_donations.loc[this_year_donations['donor_name'] == name]['amount'].sum()
        last_year_sum = last_year_donations.loc[last_year_donations['donor_name'] == name]['amount'].sum()
        year_to_year = (this_year_sum, last_year_sum)
        if this_year_sum > last_year_sum:  # upgrade if increased
            report[this_year]['upgrades'][name] = year_to_year
        else:  # no increase is a downgrade
            report[this_year]['downgrades'][name] = year_to_year

for year in report:
    print(year)
    print(f'lost: {len(report[year]["lost"])}')
    print(f'new: {len(report[year]["new"])}')
    print(f'returning: {len(report[year]["returning"])}')
    print(f'\tupgrades:   {len(report[year]['upgrades']):>4} donors for ${sum(y2y[0] - y2y[1] for name, y2y in report[year]['upgrades'].items()):9.2f}')
    print(f'\tdowngrades: {len(report[year]['downgrades']):>4} donors for ${sum(y2y[0] - y2y[1] for name, y2y in report[year]['downgrades'].items()):9.2f}')

2023
lost: 9
new: 19
returning: 301
	upgrades:    182 donors for $ 22533.53
	downgrades:  119 donors for $-11807.13
2024
lost: 10
new: 11
returning: 310
	upgrades:    185 donors for $ 22526.59
	downgrades:  125 donors for $-13247.90
2025
lost: 38
new: 8
returning: 283
	upgrades:     74 donors for $  6191.21
	downgrades:  209 donors for $-31272.77


# Year-centric analysis (aggregations)

In [8]:
year_report = pd.DataFrame(
    0., index=years, 
    columns=[
        'total', 'change',
        'num_donors', 'num_continuing',
        'donation_upgrade', 'num_upgrades', 
        'donation_downgrade', 'num_downgrades',
        'new', 'num_new',
        'lost', 'num_lost',
    ],
)

year_report['total'] = donations[['year', 'amount']].groupby('year').sum()

agg_columns = ['year', 'donor_name', 'amount']
year_name_groups = donations[agg_columns].groupby(['year', 'donor_name'])
year_donor_sum = year_name_groups.sum()

for year in years:
    this_year_donors = set(year_donor_sum.loc[year].index)
    if year == years[0]:
        # Need to insert an empty year so there's something to report in the 
        #first year we have data
        year_donor_sum.loc[year - 1, 'amount'] = 0
        year_report.loc[year, 'change'] = year_report.loc[year, 'total']
    else:    
        year_report.loc[year, 'change'] = (
            year_report.loc[year, 'total'] - year_report.loc[year - 1, 'total']
        )

    y2y_change = year_donor_sum.loc[year] - year_donor_sum.loc[year - 1]

    last_year_donors = set(year_donor_sum.loc[year - 1].index)
    last_year_donors.discard('')  # injected empty year has donor ''
    new = this_year_donors - last_year_donors
    lost = last_year_donors - this_year_donors
    continuing = last_year_donors & this_year_donors
    year_report.loc[year, 'num_donors'] = len(this_year_donors)

    year_report.loc[year, 'num_new'] = len(new)
    year_report.loc[year, 'new'] = year_donor_sum.loc[year].loc[sorted(new)].sum().item()

    year_report.loc[year, 'num_lost'] = len(lost)
    year_report.loc[year, 'lost'] = year_donor_sum.loc[year - 1].loc[sorted(lost)].sum().item()
    
    year_report.loc[year, 'num_continuing'] = len(continuing)

    num_upgrades = (y2y_change > 0).sum().item()
    year_report.loc[year, 'num_upgrades'] = num_upgrades
    
    upgrade = y2y_change[y2y_change > 0].sum().item()
    year_report.loc[year, 'donation_upgrade'] = upgrade

    num_downgrades = (y2y_change <= 0).sum().item()
    year_report.loc[year, 'num_downgrades'] = num_downgrades
    
    downgrade = y2y_change[y2y_change <= 0].sum().item()
    year_report.loc[year, 'donation_downgrade'] = downgrade

    if year == 2025:
        print(new)
    

year_report['num_upgrades'] = year_report['num_upgrades'].astype(int)
year_report['num_downgrades'] = year_report['num_downgrades'].astype(int)
year_report['num_donors'] = year_report['num_donors'].astype(int)
year_report['num_new'] = year_report['num_new'].astype(int)
year_report['num_lost'] = year_report['num_lost'].astype(int)
year_report['num_continuing'] = year_report['num_continuing'].astype(int)

display(year_report)

{'Ana Lopez', 'Miguel Taylor', 'Juan Ramirez', 'Hannah Brown', 'Luis Wilson', 'Isabel Torres', 'Gabriela Sanchez', 'Jose Brown'}


,total,change,num_donors,num_continuing,donation_upgrade,num_upgrades,donation_downgrade,num_downgrades,new,num_new,lost,num_lost
2022,41713.10,41713.10,310,0,0.00,0,0.00,0,41713.10,310,0.00,0
2023,54102.84,12389.74,320,301,22533.53,182,-11807.13,119,2623.54,19,960.20,9
2024,64278.58,10175.74,321,310,22526.59,185,-13247.90,125,1837.54,11,940.49,10
2025,33626.86,-30651.72,291,283,6191.21,74,-31272.77,209,632.77,8,6202.93,38


# Donor-centric Analysis

In [9]:
REPORT_YEAR = 2025

donor_stats_columns = ['num_years_donated', 'first_year', 'last_year', 'status', 'average_annual_donation']
donor_stats = pd.DataFrame(index=sorted(donations['donor_name'].unique()), columns=donor_stats_columns)
donor_stats.index.name = 'donor_name'

donor_year_amounts = donations[agg_columns].groupby(['donor_name', 'year']).sum().reset_index(['year'])

donor_years = donor_year_amounts.reset_index()[['donor_name', 'year']].groupby('donor_name')
donor_stats['num_years_donated'] = donor_years.count()
donor_stats['first_year'] = donor_years.min()
donor_stats['last_year'] = donor_years.max()

donor_stats['average_annual_donation'] = donor_year_amounts.reset_index().drop(columns=['year']).groupby('donor_name').mean()

for donor in donor_stats.index:
    this_donor_mask = donor_year_amounts.index == donor  # use a mask to always return a DataFrame
    this_donor_year_amounts = donor_year_amounts[this_donor_mask].set_index('year').sort_index()['amount']
    if donor in {'Isabel Torres', 'Gabriela Sanchez', 'Ana Lopez', 'Hannah Brown', 'Juan Ramirez', 'Luis Wilson', 'Miguel Taylor', 'Jose Brown'}:
        # These donors show up as 'New' in other analysis
        print(donor, this_donor_year_amounts)
    
    if this_donor_year_amounts.index.max() != REPORT_YEAR:
        status = 'Lost'
    elif this_donor_year_amounts.index.min() == REPORT_YEAR:
        status = 'New'
    else:
        if this_donor_year_amounts.iloc[-1] > this_donor_year_amounts.iloc[-2]:
            status = 'Upgrade'
        else:
            status = 'Downgrade'
    donor_stats.loc[donor, 'status'] = status
            
display(donor_stats.head())

Ana Lopez year
2022    18.35
2023    19.58
2025    65.51
Name: amount, dtype: float64
Gabriela Sanchez year
2022    57.99
2023    75.20
2025    42.25
Name: amount, dtype: float64
Hannah Brown year
2022     22.34
2023    315.75
2025    172.41
Name: amount, dtype: float64
Isabel Torres year
2022    179.38
2023     83.73
2025     18.92
Name: amount, dtype: float64
Jose Brown year
2022    106.10
2023     29.04
2025     99.79
Name: amount, dtype: float64
Juan Ramirez year
2022    132.65
2023     69.87
2025     41.73
Name: amount, dtype: float64
Luis Wilson year
2022    119.01
2023    119.69
2025    122.84
Name: amount, dtype: float64
Miguel Taylor year
2023    121.01
2025     69.32
Name: amount, dtype: float64


,num_years_donated,first_year,last_year,status,average_annual_donation
donor_name,,,,,
Adrian Hernandez,4,2022,2025,Downgrade,127.2025
Adrian Johnson,4,2022,2025,Downgrade,222.4125
Adrian Perez,4,2022,2025,Downgrade,106.3825
Adrian Taylor,4,2022,2025,Upgrade,98.7500
Adrian Torres,3,2022,2024,Lost,275.8800


In [10]:
donor_stats.reset_index().set_index(['status', 'donor_name']).sort_index()

num_years_donated  first_year  last_year  \
status    donor_name                                                   
Downgrade Adrian Hernandez                  4        2022       2025   
          Adrian Johnson                    4        2022       2025   
          Adrian Perez                      4        2022       2025   
          Ana Brown                         4        2022       2025   
          Ana Davis                         4        2022       2025   
...                                       ...         ...        ...   
Upgrade   Susan Gomez                       4        2022       2025   
          Tim Flores                        4        2022       2025   
          Tim Johnson                       3        2022       2025   
          Tim Ramirez                       3        2023       2025   
          Tim Taylor                        4        2022       2025   

                            average_annual_donation  
status    donor_name                                 
Downgrade Adrian Hernandez               127.202500  
          Adrian Johnson                 222.412500  
          Adrian Perez                   106.382500  
          Ana Brown                       71.585000  
          Ana Davis                      163.530000  
...                                             ...  
Upgrade   Susan Gomez                    271.680000  
          Tim Flores                     284.672500  
          Tim Johnson                    135.713333  
          Tim Ramirez                    162.896667  
          Tim Taylor                      97.947500  

[331 rows x 4 columns]